In [ ]:
import numpy as np
import pandas as pd

# Step 1: Load Dataset (Simulasi Data untuk Beasiswa PIP)
data = {
    "Nama": ["Siswa A", "Siswa B", "Siswa C", "Siswa D"],
    "Penghasilan_Ayah": [2000000, 1500000, 3000000, 1000000],
    "Penghasilan_Ibu": [1500000, 1200000, 2500000, 800000],
    "Jumlah_Saudara": [3, 2, 4, 5],
    "Pekerjaan_Ayah": [2, 3, 1, 2],  # 1: Tidak Bekerja, 2: Buruh, 3: PNS
    "Pekerjaan_Ibu": [1, 2, 2, 1],  # 1: Ibu Rumah Tangga, 2: Buruh, 3: PNS
}
df = pd.DataFrame(data)

# Step 2: Define Fuzzy Membership Functions
def fuzzy_penghasilan(value):
    """Fungsi keanggotaan untuk penghasilan (dalam rupiah)."""
    if value <= 1500000:
        return 1.0  # Rendah
    elif 1500000 < value <= 3000000:
        return (3000000 - value) / 1500000  # Sedang
    else:
        return 0.0  # Tinggi

def fuzzy_jumlah_saudara(value):
    """Fungsi keanggotaan untuk jumlah saudara."""
    if value >= 4:
        return 1.0  # Banyak
    elif 2 <= value < 4:
        return (value - 2) / 2  # Sedang
    else:
        return 0.0  # Sedikit

def fuzzy_pekerjaan(value):
    """Fungsi keanggotaan untuk pekerjaan."""
    if value == 1:
        return 1.0  # Tidak Bekerja / Ibu Rumah Tangga
    elif value == 2:
        return 0.5  # Buruh
    else:
        return 0.0  # PNS atau lainnya

# Step 3: Apply Fuzzy Membership Functions to Dataset
fuzzy_data = []
for _, row in df.iterrows():
    fuzzy_row = {
        "Nama": row["Nama"],
        "Fuzzy_Penghasilan_Ayah": fuzzy_penghasilan(row["Penghasilan_Ayah"]),
        "Fuzzy_Penghasilan_Ibu": fuzzy_penghasilan(row["Penghasilan_Ibu"]),
        "Fuzzy_Jumlah_Saudara": fuzzy_jumlah_saudara(row["Jumlah_Saudara"]),
        "Fuzzy_Pekerjaan_Ayah": fuzzy_pekerjaan(row["Pekerjaan_Ayah"]),
        "Fuzzy_Pekerjaan_Ibu": fuzzy_pekerjaan(row["Pekerjaan_Ibu"]),
    }
    fuzzy_data.append(fuzzy_row)

fuzzy_df = pd.DataFrame(fuzzy_data)

# Step 4: Define Weights for SAW (Bobot Kriteria)
weights = {
    "Fuzzy_Penghasilan_Ayah": 0.3,
    "Fuzzy_Penghasilan_Ibu": 0.3,
    "Fuzzy_Jumlah_Saudara": 0.2,
    "Fuzzy_Pekerjaan_Ayah": 0.1,
    "Fuzzy_Pekerjaan_Ibu": 0.1,
}

# Step 5: Calculate SAW Scores
fuzzy_df["SAW_Score"] = fuzzy_df.apply(
    lambda row: sum(row[criteria] * weight for criteria, weight in weights.items()), axis=1
)

# Step 6: Rank Students Based on SAW Scores
fuzzy_df["Rank"] = fuzzy_df["SAW_Score"].rank(ascending=False)

# Step 7: Save Model for Deployment
fuzzy_df.to_csv("fuzzy_saw_results.csv", index=False)

# Display Results
print("Final Results with SAW Scores:")
print(fuzzy_df.sort_values(by="Rank"))


Final Results with SAW Scores:
      Nama  Fuzzy_Penghasilan_Ayah  Fuzzy_Penghasilan_Ibu  \
3  Siswa D                1.000000               1.000000   
0  Siswa A                0.666667               1.000000   
1  Siswa B                1.000000               1.000000   
2  Siswa C                0.000000               0.333333   

   Fuzzy_Jumlah_Saudara  Fuzzy_Pekerjaan_Ayah  Fuzzy_Pekerjaan_Ibu  SAW_Score  \
3                   1.0                   0.5                  1.0       0.95   
0                   0.5                   0.5                  1.0       0.75   
1                   0.0                   0.0                  0.5       0.65   
2                   1.0                   1.0                  0.5       0.45   

   Rank  
3   1.0  
0   2.0  
1   3.0  
2   4.0  
